<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-31 20:46:11--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8175 (8.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.98K  --.-KB/s    in 0s      

2022-05-31 20:46:12 (108 MB/s) - ‘utils.py’ saved [8175/8175]



In [3]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [4]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x3/1'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [8]:
classes, class_weight = get_class_weight()

In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [11]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')

In [12]:
hsv_model = hsv_creator.make_model()
callbacks = hsv_creator.get_callbacks()

In [13]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=5,
                        class_weight=class_weight)

Epoch 1/5
519/519 [==============================] - 2472s 5s/step - loss: 2382.4980 - precision: 0.4737 - recall: 0.4734 - auc: 0.2976 - accuracy: 0.4734
Epoch 2/5
519/519 [==============================] - 2419s 5s/step - loss: 1657.7479 - precision: 0.5185 - recall: 0.5185 - auc: 0.3405 - accuracy: 0.5185
Epoch 3/5
519/519 [==============================] - 2418s 5s/step - loss: 1079.3662 - precision: 0.5639 - recall: 0.5638 - auc: 0.3895 - accuracy: 0.5638
Epoch 4/5
 22/519 [>.............................] - ETA: 38:38 - loss: 1096.9510 - precision: 0.5619 - recall: 0.5611 - auc: 0.3898 - accuracy: 0.5611

KeyboardInterrupt: ignored

In [15]:
hsv_model.save_weights('drive/MyDrive/checkpoints/hsv-weights.h5')